In [1]:
from lesson3Code.pokemonTournament.pokemonDef import Monster,superEffective,notEffective,noEffect
import lesson3Code.pokemonTournament.pokemonFunctions as pf
import sys, random, time, math
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

Modelling
- first hour
    - experimentral alg
    - pipeline
    - examples
    - homework
- second hour
    - pokemon
    - intro I.P

ゴール： 現実的な問題をコンピュータの能力内で解く  
JEA SEAなどのコミュニティサイトがある.  
論文をまとめてあるサイト？gatech?なども使える．  
アルゴリズムを論文に乗せるのに大事なこと，実験に大事なこと？  
コードを見せることではなく，問題がなんなのか，その解法の長所と短所はなんなのか，にたいする洞察を示すことが大事である．  

# EA pipeline Step  
<br>
<div style="text-align:center">problem(client) -> formalization -> algorithm design -> implementation -> experimentation</div>
<br>
この一連のステップそれぞれからPractical Insightを導き，さらにFeedbackを付け加える  
homeworkはexperimentationの辺りから何か出される

## Formalization
needs storong mathematical foundations  
理論上重要なことが実践では重要ではないことがある．  
Bias filtering vs Approximation algorithm

## Algorithm design
どんな点でBest solutionなのか?(最適解を出せるのか，早いのか)  
実装時に発生する問題を考慮してデザインすることが大事

## Implementation
Programming language, team, task distribution...  
専門知識なしで，誰が実装しても良いようにしたい．  
実装時間 vs 実行時間

## Experiments
実験の設計(何をどうやって測定するのか)を定める  
実験時の「バイアス」に注意する  
- 認証バイアス(confirmation bias)世の中で正しいとされているもの，正しいと自分が信じているものを正しいとしてしまうこと
- 文化バイアス(culture bias)メディアなど？

## Insight
- 洞察の種類(問題，アプローチ)
    - 結果の解釈
    - 仮説検証
- SOFT skills

# Practical Examples
- road network matching
- medical image analysis
- 3D point set residues

## Road network matching
地図の一部がどこのものであるかを調べる  
交差点と交差点への彩色問題によって，$O(n^4 n'^{3} \log n')$で解ける. YAGO先生の論文より

## Medical Image Analysis

## 3D Point Cloud Matching
複数の角度から撮ったうさぎの3Dモデルができるって話  
最低3つの同じ位置にある点を見つける必要がある．  $O(n^6)$  
最新では$O(n^2)$くらい？
最悪計算量のオーダが多くても，現実のデータで例えば全てのなんらかの対象となるピクセルがある領域に固まっていることはないなどの仮定を置けば有用なアルゴリズムとなる．

# Toy Problem
which is the strongest Pokemon?
Formalization
- how many pokemon are there?
    251? FIRST GENERATION Pokemons.
- what does "strong" mean?
    winners of Combats
- read bibliograohy
    - 参考：https://hubpages.com/entertainment/Top-10-Strongest-First-Generation-Pokemons

Algorithm design
- combat system はユーザのコマンドを必要とする
- タイプシステム
- Moves can be developed using points during the game
- Official codeを入手することはできない

## Implementation
easy to implement  
no money  
python  

Home work
1. review the python code, feel free to improve it.
2. design experiment(s) to find out the stronger pokemon and pokemon team
    team: 6 pokemon, 1匹ずつ戦い，勝ったほうが次のポケモンと戦う
3. Report
    - results/comments insight!
    - Given the restrictions, how would you improve the process(Formalization, design, implementation)
    - restrictionsがなければどうやってimproveするか？
    - 結果は予想どうりのものか？そうでなければ何が問題だったのか
    - もうなんかよくわかんないので，最強のポケモンとポケモンチームを得るために何をやったのかまとめて，あと最後にちょっとした考察を付け加えることにする

とりあえず思いつくのは総当たり戦をして，勝った回数が一番多いやつが強い  
より早くしたいならランダムリーグ戦をやらせる．これは近似解になる．(タイプシステムがあるので最強とは限らない)

In [2]:
listOfNamesAndTypes=pf.readPokemonTypesFromFile("lesson3Code/pokemonTournament/listOfPokemon1.txt")
pokeList=pf.readPokemonListFromFile("lesson3Code/pokemonTournament/listOfPokemon2.txt")

pokedex={}
for monster in pokeList:
    pokedex[monster.name]=monster

for name,listOfTypes in listOfNamesAndTypes:
    for t in listOfTypes:
        pokedex[name].variety.append(t)

In [3]:
def RoundRobinExperiment(k=1, battle=pf.battle, result=True):
    win_times = np.zeros(len(pokeList), dtype='i')
    for i in range(k):
        for player in range(len(pokeList)):
            win_time = 0
            for enemy in range(len(pokeList)):
                if player == enemy: continue
                winner = battle(pokeList[player], pokeList[enemy])
                if winner == 0: win_time += 1
            win_times[player] += win_time

    if result:
        strongest = win_times.argmax()
        print("The strongest pokemon is", pokeList[strongest].name, "id: ", strongest)
        print("The strongest pokemon parameter", pokeList[strongest])
        print("Strongest Pokemon Top 10")
        for i in win_times.argsort()[::-1][:10]:
            print(pokeList[i])
    
    return win_times

総当たり: $O(n^2)$  
$K$回の総当たり: $O(Kn^2)$  
この場合は，「あるポケモンが自分より弱いポケモンが何種類いるか」が強さの基準になっている．  
- 長所
    - ポケモンの能力値がわからなくても強さを測定できる．
    - 実装が簡単である
- 短所
    - 時間がかかる
    - Combat Systemに乱数(一様分布)が利用されているため,実験の度に結果が変わる
        - 何回繰り返したところで最強とは限らない

仮説
- 同じタイプの場合，攻撃力，防御力，HPで力関係の期待値がわかる
    - 毎ターンそれぞれが対決
    - 攻撃力と防御力の変動値は同じタイプであれば一様分布で決まる

In [4]:
def SpeedBattle(p1, p2, resetHP=True, verbose=False):
    effectiveness = 0.3
    max_rounds = 1000
    
    if resetHP:
        pf.ResetHP(p1)
        pf.ResetHP(p2)
    
    if noEffect(p1, p2) and noEffect(p2, p1): return 2
    elif noEffect(p1, p2): return 1
    elif noEffect(p2, p1): return 0
    
    attackCenter1=1
    if superEffective(p1, p2): attackCenter1+=effectiveness
    elif notEffective(p1, p2): attackCenter1-=effectiveness

    attackCenter2=1
    if superEffective(p2, p1): attackCenter2+=effectiveness
    elif notEffective(p2, p1): attackCenter2-=effectiveness
    
    get_damage1 = max(0, (round(p2.attack * attackCenter2) - round(p1.defense * 0.9)))
    survive_rounds1 = (p1.hitpoint[1] / get_damage1) if get_damage1 > 0 else max_rounds
        
    get_damage2 = max(0, (round(p1.attack * attackCenter1) - round(p2.defense * 0.9)))
    survive_rounds2 = (p2.hitpoint[1] / get_damage2) if get_damage2 > 0 else max_rounds
        
    if survive_rounds1 == survive_rounds2: 
        if survive_rounds1 == max_rounds: return 2
        if p1.speed == p2.speed: return 2
        return 0 if p1.speed > p2.speed else 1
    
    return 0 if survive_rounds1 > survive_rounds2 else 1

保存版最強リスト(K=100)  
The strongest pokemon is Cloyster id:  90  
The strongest pokemon parameter Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])  
Strongest Pokemon Top 10  
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])  
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])  
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, -25], variety=['Water', 'Flying'])  
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 36], variety=['Psychic'])  
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 71], variety=['Dragon', 'Flying'])  
Monster(name='Mew', index=150, attack=100, defense=100, speed=100, hitpoint=[100, -1], variety=['Psychic'])  
Monster(name='Kabutops', index=140, attack=115, defense=105, speed=70, hitpoint=[60, -8], variety=['Rock', 'Water'])  
Monster(name='Pinsir', index=126, attack=125, defense=100, speed=55, hitpoint=[65, -1], variety=['Bug'])  
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])  
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, -5], variety=['Fighting'])  
====================== Speed Battle =========================  
The strongest pokemon is Kingler id:  98  
The strongest pokemon parameter Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])  
Strongest Pokemon Top 10  
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])  
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 95], variety=['Water', 'Flying'])  
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])  
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])  
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], variety=['Fighting'])  
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 106], variety=['Psychic'])  
Monster(name='Pinsir', index=126, attack=125, defense=100, speed=55, hitpoint=[65, 65], variety=['Bug'])  
Monster(name='Mew', index=150, attack=100, defense=100, speed=100, hitpoint=[100, 100], variety=['Psychic'])  
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])  
Monster(name='Kabutops', index=140, attack=115, defense=105, speed=70, hitpoint=[60, 60], variety=['Rock', 'Water'])  

In [5]:
# K = 100
# normal_result = RoundRobinExperiment(K)
normal_ranking = [
    Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice']),
    Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water']),
    Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, -25], variety=['Water', 'Flying']),
    Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 36], variety=['Psychic']),
    Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 71], variety=['Dragon', 'Flying']),
    Monster(name='Mew', index=150, attack=100, defense=100, speed=100, hitpoint=[100, -1], variety=['Psychic']),
    Monster(name='Kabutops', index=140, attack=115, defense=105, speed=70, hitpoint=[60, -8], variety=['Rock', 'Water']),
    Monster(name='Pinsir', index=126, attack=125, defense=100, speed=55, hitpoint=[65, -1], variety=['Bug']),
    Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock']),
    Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, -5], variety=['Fighting']),
]

In [6]:
print("====================== Speed Battle =========================")
speed_result = RoundRobinExperiment(battle=SpeedBattle)

====================== Speed Battle =========================
The strongest pokemon is Kingler id:  98
The strongest pokemon parameter Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Strongest Pokemon Top 10
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 95], variety=['Water', 'Flying'])
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], variety=['Fighting'])
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 106], variety=['Psychic'])
Monster(name='Pinsir', index=126, attack=125, defens

In [7]:
# same_places = normal_result.argsort() == speed_result.argsort()
# print(same_places)
# print(f"same places: {sum(same_places)} / {len(same_places)}")

順位が同じ，という基準では少し厳しい気がするので，レーベンシュタイン距離を基準にする

In [8]:
# Levenstein Distance by Dynamic Programming
def LevensteinDistance(l1, l2):
    len1 = len(l1) + 1
    len2 = len(l2) + 1
    
    dp = np.zeros((len1, len2), dtype='i')
    for row in range(len1): dp[row][0] = row
    for col in range(len2): dp[0][col] = col
        
    for row in range(1, len1):
        for col in range(1, len2):
            c = 0 if l1[row-1] == l2[col-1] else 1
            dp[row][col] = min(dp[row-1][col]+1, dp[row][col-1]+1, dp[row-1][col-1] + c)

    return dp[-1][-1]

In [9]:
# LevensteinDistance(normal_result.argsort(), speed_result.argsort())

よって，あんまり変わらない

In [10]:
startNormal = time.time()
RoundRobinExperiment(result=False)
endNormal = time.time()
print("Normal Battle Elapsed Time:", endNormal - startNormal)

startSpeed = time.time()
RoundRobinExperiment(battle=SpeedBattle, result=False)
endSpeed = time.time()
print("Speed Battle Elapsed Time:", endSpeed - startSpeed)

Normal Battle Elapsed Time: 8.003353834152222
Speed Battle Elapsed Time: 0.46237897872924805


In [11]:
strongest = normal_ranking[0]
print(strongest)

max_attack = max([p.attack for p in pokeList])

print("Enemy of Cloyster at leaste needs to be SuperEffective variety, because max_attack * 1.2 = ", max_attack * 1.2, "<", strongest.defense)

Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Enemy of Cloyster at leaste needs to be SuperEffective variety, because max_attack * 1.2 =  160.79999999999998 < 180


In [12]:
print("Pokemon Defense Ranking")
defense_ranking = np.argsort([p.defense for p in pokeList])[::-1]
for p in defense_ranking[:10]:
    print(pokeList[p])

Pokemon Defense Ranking
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Onix', index=94, attack=45, defense=160, speed=30, hitpoint=[35, 35], variety=['Rock', 'Ground'])
Monster(name='Golem', index=75, attack=110, defense=130, speed=55, hitpoint=[80, 80], variety=['Rock', 'Ground', 'Electric'])
Monster(name='Omastar', index=138, attack=60, defense=125, speed=115, hitpoint=[70, 70], variety=['Rock', 'Water'])
Monster(name='Weezing', index=109, attack=90, defense=120, speed=85, hitpoint=[65, 65], variety=['Poison'])
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])
Monster(name='Graveler', index=74, attack=95, defense=115, speed=45, hitpoint=[55, 55], variety=['Rock', 'Ground', 'Electric'])
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Tangela', index=113, attack=55, d

In [13]:
print("Pokemon Attack Ranking")
attack_ranking = np.argsort([p.attack for p in pokeList])[::-1]
for p in attack_ranking[:10]:
    print(pokeList[p])

Pokemon Attack Ranking
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], variety=['Fighting'])
Monster(name='Flareon', index=135, attack=130, defense=60, speed=110, hitpoint=[65, 65], variety=['Fire'])
Monster(name='Pinsir', index=126, attack=125, defense=100, speed=55, hitpoint=[65, 65], variety=['Bug'])
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 95], variety=['Water', 'Flying'])
Monster(name='Hitmonlee', index=105, attack=120, defense=53, speed=35, hitpoint=[50, 50], variety=['Fighting'])
Monster(name='Kabutops', index=140, attack=115, defense=105, speed=70, hitpoint

In [14]:
round(180 * 0.7)

126

Effective pokemons should have attack > 126  
Or who have varieties against Water and Ice are...
- Grass
- Electric
- Fighting
- Rock
- Steel

and have enough attack

In [15]:
effective_varieties = ['Grass', 'Electric', 'Fighting', 'Rock', 'Steel']
effective_pokeList = [p for p in pokeList if np.any([v in p.variety for v in effective_varieties]) or p.attack > 126]
print("# of Pokemons who can fight Cloyster:", len(effective_pokeList), "/", len(pokeList))

winnable_pokeList = [p for p in effective_pokeList if p.attack * 1.3 * 1.2 > 180 * 0.7]
print("\n# of Pokemons who can give a damage to Cloyster:", len(winnable_pokeList), "/", len(pokeList))

for p in winnable_pokeList:
    print(p)

# of Pokemons who can fight Cloyster: 49 / 151

# of Pokemons who can give a damage to Cloyster: 24 / 151
Monster(name='Venusaur', index=2, attack=82, defense=83, speed=100, hitpoint=[80, 80], variety=['Grass', 'Poison'])
Monster(name='Raichu', index=25, attack=90, defense=55, speed=90, hitpoint=[60, 60], variety=['Electric', 'Psychic'])
Monster(name='Sandslash', index=27, attack=100, defense=110, speed=55, hitpoint=[75, 75], variety=['Ground', 'Steel'])
Monster(name='Parasect', index=46, attack=95, defense=80, speed=80, hitpoint=[60, 60], variety=['Bug', 'Grass'])
Monster(name='Primeape', index=56, attack=105, defense=60, speed=60, hitpoint=[65, 65], variety=['Fighting'])
Monster(name='Poliwrath', index=61, attack=85, defense=95, speed=70, hitpoint=[90, 90], variety=['Water', 'Fighting'])
Monster(name='Machoke', index=66, attack=100, defense=70, speed=50, hitpoint=[80, 80], variety=['Fighting'])
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], var

In [16]:
K = 1000
cloyster = pokedex["Cloyster"]
challengers = winnable_pokeList

win_rates = []
for challenger in challengers:
    beat = 0
    for i in range(K):
        result = pf.battle(cloyster, challenger)
        if result == 1: beat += 1
    win_rates.append([challenger, beat])
win_rates = np.array(win_rates)
    
ranking = win_rates[np.argsort(np.array(win_rates)[:, 1])][::-1]
for challenger, beat in ranking:
    print(f"Challenger {challenger.name} Win Rate: {beat}/{K}")

Challenger Rhydon Win Rate: 728/1000
Challenger Machamp Win Rate: 457/1000
Challenger Golem Win Rate: 118/1000
Challenger Dragonite Win Rate: 42/1000
Challenger Victreebel Win Rate: 1/1000
Challenger Hitmonlee Win Rate: 1/1000
Challenger Weepinbell Win Rate: 0/1000
Challenger Raichu Win Rate: 0/1000
Challenger Sandslash Win Rate: 0/1000
Challenger Parasect Win Rate: 0/1000
Challenger Primeape Win Rate: 0/1000
Challenger Poliwrath Win Rate: 0/1000
Challenger Machoke Win Rate: 0/1000
Challenger Aerodactyl Win Rate: 0/1000
Challenger Flareon Win Rate: 0/1000
Challenger Electabuzz Win Rate: 0/1000
Challenger Graveler Win Rate: 0/1000
Challenger Zapdos Win Rate: 0/1000
Challenger Kingler Win Rate: 0/1000
Challenger Exeggutor Win Rate: 0/1000
Challenger Hitmonchan Win Rate: 0/1000
Challenger Rhyhorn Win Rate: 0/1000
Challenger Kabutops Win Rate: 0/1000
Challenger Venusaur Win Rate: 0/1000


この結果からわかるように，パルシェンにはサイドンをぶつけるのが有効である．  
ちなみにSpeedBattleのようにパラメータで勝負させるとみんな負ける．

# ポケモンチーム戦
前提
- 1チームは6匹のポケモンで構成される
- 勝ったポケモンが次のポケモンと戦う
- 引き分けの場合，両者敗北とし，それぞれ次のポケモンを戦わせる
- 先に相手のポケモン全員に勝った方の勝ち

制約
- 同じポケモンを編成可能かどうか
- 相手のチームがわかっているか

In [19]:
def team_battle(team1, team2, battle_num=1, battle=pf.battle, verbose=False, run_first=True):
    team_pokemon_num = 6
    
    team1 = team1.copy()
    team2 = team2.copy()
    
    run_first_limit = math.ceil(battle_num / 2)
    
    # reset HP
    for p in range(team_pokemon_num):
        pf.ResetHP(team1[p])
        pf.ResetHP(team2[p])
    
    if verbose:
        print("\n======== Team Battle =======\n")
        print("Team1")
        for p in range(team_pokemon_num): print(team1[p])
        print()
        print("Team2")
        for p in range(team_pokemon_num): print(team2[p])
        print()
    
    team1_win = 0
    team2_win = 0
    
    for i in range(battle_num):
        if verbose:
            print(f"\n======= Battle {i + 1} =======\n")
        
        # reset HP
        for p in range(team_pokemon_num):
            pf.ResetHP(team1[p])
            pf.ResetHP(team2[p])
        
        team1_remain = team_pokemon_num
        team2_remain = team_pokemon_num
        
        # Team Battle Start
        while team1_remain > 0 and team2_remain > 0:
            p1 = team1[-team1_remain]
            p2 = team2[-team2_remain]
            
            if verbose:
                print(f"{p1.name} vs {p2.name}")
                print(p1)
                print(p2)
            
            result = battle(p1, p2, resetHP=False, verbose=False)
            if result == 0: 
                team2_remain -= 1
                if verbose:
                    print(f"team1 {p1.name} win")
            elif result == 1: 
                team1_remain -= 1
                if verbose:
                    print(f"team2 {p2.name} win")
            else:
                team1_remain -=1
                team2_remain -=1
                if verbose:
                    print("Draw")
            
            if verbose:
                print(f"Team1 remain: {team1_remain}")
                print(f"Team2 remain: {team2_remain}")
                print()
        
        if team1_remain > team2_remain:
            team1_win += 1
            if verbose: print("Team1 win")
        elif team1_remain < team2_remain:
            team2_win += 1
            if verbose: print("Team2 win")
        else :
            if verbose: print("Draw")
        
        if run_first and (team1_win >= run_first_limit or team2_win >= run_first_limit):
            if verbose: print(f"{run_first_limit} battles run first")
            break
    
    return (team1_win, team2_win)

In [20]:
team1 = [pokeList[p] for p in np.random.randint(0, len(pokeList),6)]
team2 = [pokeList[p] for p in np.random.randint(0, len(pokeList),6)]

team_battle(team1, team2, battle_num=100, verbose=False)

(3, 50)

TeamBattleができるようになったところで最強チームを決めたい．  
とりあえず最強チームは総当たりランキング上位6匹にしてみる

In [21]:
strongest_team = [pokeList[p] for p in speed_result.argsort()[::-1][:6]]

こいつがどれだけ他のチームより強いか確かめたいが，チームの組み合わせは100兆を超えるので，総当たりは現実的でない．  
組み合わせでも100億=10^10

In [22]:
from itertools import permutations
all_teams = permutations(range(len(pokeList)), r=6)
151 ** 6

11853911588401

組み合わせだと100億=10^10  
これも現実的でない

In [23]:
from itertools import combinations
all_teams = combinations(range(len(pokeList)), r=1)
151*150*149*148*147*146 / (6*5*4*3*2)

14888600755.0

とりあえずランダムで戦わせてみる

In [24]:
import random

def random_battle(team, challenger_team_num=1000, battle_num=10):
    winners = []
    for i in range(challenger_team_num):
        challenger_team = [pokeList[p] for p in random.sample(range(len(pokeList)),6)]
        (team_win, challenger_win) = team_battle(team, challenger_team, battle_num=battle_num)
        if team_win < challenger_win: 
            winners.append(challenger_team.copy())
    
    return winners

In [25]:
winners = random_battle(strongest_team, challenger_team_num=10000)
print(f"Challenger Win Rate: {len(winners)} / 10000")
print()
for w in winners:
    for p in w: print(p)
    print()

Challenger Win Rate: 2 / 10000

Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], variety=['Fighting'])
Monster(name='Snorlax', index=142, attack=110, defense=65, speed=65, hitpoint=[160, -43], variety=['Normal'])
Monster(name='Dodrio', index=84, attack=110, defense=70, speed=60, hitpoint=[60, -11], variety=['Normal', 'Flying'])
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Bellsprout', index=68, attack=75, defense=35, speed=70, hitpoint=[50, -30], variety=['Grass', 'Poison'])
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 106], variety=['Psychic'])

Monster(name='Hypno', index=96, attack=73, defense=70, speed=115, hitpoint=[85, -18], variety=['Psychic'])
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])
Monster(name='Venomoth', index=48, attack=65, defense=60, s

In [26]:
if len(winners) > 0:
    team_battle(strongest_team, winners[0], verbose=True)


======== Team Battle =======

Team1
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 95], variety=['Water', 'Flying'])
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], variety=['Fighting'])
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 106], variety=['Psychic'])

Team2
Monster(name='Machamp', index=67, attack=130, defense=80, speed=65, hitpoint=[90, 90], variety=['Fighting'])
Monster(name='Snorlax', index=142, attack=110, defense=65, speed=65, hitpoint=[160, 160], variety=['Normal'])
Monster(name='Dodrio', index=84, attack=110, defense=70,

この戦いから，Cloysterに対抗できるのはサイドンだけでないことが分かる．  
Cloysterからダメージを受けないSlowbroなどのポケモンは，ダメージこそ与えられないがCloysterとドローすることができ，Cloysterを退けられる．  
では，Cloysterが前半でやられないよう，最後に持ってくるのはどうか

In [27]:
reversed_strongest_team = [strongest_team[i] for i in [1,2,3,4,5,0]]

winners = random_battle(reversed_strongest_team)
print(f"Challenger Win Rate: {len(winners)} / 1000")
print()
for w in winners:
    for p in w: print(p)
    print()

Challenger Win Rate: 0 / 1000



In [28]:
if len(winners) > 0:
    team_battle(reversed_strongest_team, winners[0], verbose=True)

順番変えても負けるもんは負けた

TeamTrainingアルゴリズム

1. 最初のチームをランダムで作り，Tとする
2. チームTがK回連勝するまで次を繰り返す
    1. Tがランダムバトルを行う
    2. ランダムバトルでTが負けた場合
        1. Tに勝ったチームをTとする
        2. 連勝カウントをリセットする
3. Tを出力する


- 良いところ
    - リーズナブルな時間で解が求まる
    - どんなバトルのルールにも適用できる
    - 実装が簡単でシンプル
- 悪いところ
    - 本当に最強かどうかはわからない
    - kの値を大きくすると時間がかかる
    - いつ終わるかわからない
        - チームはランダムに選ばれる
        - 3すくみのような状態が発生するとループし続ける
        - SpeedBattleのように，勝者に決定性があれば収束する

In [29]:
def pteam(team):
    for p in team: print(p)
    print()

def team_training(k=10, battle_num=10, battle=SpeedBattle, verbose=False):
    pokeidx = range(len(pokeList))
    team = [pokeList[p] for p in random.sample(pokeidx,6)]

    if verbose: 
        print("======== Initial Team ========")
        pteam(team)

    consecutive_win_num = 0
    team_count = 1
    while consecutive_win_num < k:
        
        rival = [pokeList[p] for p in random.sample(pokeidx,6)]
        (team_win, rival_win) = team_battle(team, rival, battle=battle, battle_num=battle_num)
        
        if team_win < rival_win: 
            team = rival

            if verbose:
                print(f"======== {team_count}-th New Team =======")
                pteam(team)

            consecutive_win_num = 0
            team_count += 1
        else: 
            consecutive_win_num += 1
    
    if verbose:
        print(f"======== {team_count-1}-th team is the strongest team ========")
        pteam(team)

    return team, team_count-1

In [30]:
start_time = time.time()
strongest_team, team_count = team_training(k=100)
end_time = time.time()

print(f"======== {team_count-1}-th team is the strongest team ========")

pteam(strongest_team)

print("Training Team Elapsed Time: ", end_time - start_time)

======== 104-th team is the strongest team ========
Monster(name='Nidoking', index=33, attack=92, defense=77, speed=75, hitpoint=[81, 81], variety=['Poison', 'Ground'])
Monster(name='Poliwrath', index=61, attack=85, defense=95, speed=70, hitpoint=[90, 90], variety=['Water', 'Fighting'])
Monster(name='Koffing', index=108, attack=65, defense=95, speed=60, hitpoint=[40, 40], variety=['Poison'])
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Snorlax', index=142, attack=110, defense=65, speed=65, hitpoint=[160, 160], variety=['Normal'])

Training Team Elapsed Time:  0.49616503715515137


TeamTrainingアルゴリズムではk=10000の場合だといつまで経っても収束しない．  
そもそもタイプの問題があるので，どんなチームにも勝てるチーム，というのは存在しない．  
例えば，3つのチームが3すくみの関係にある場合，各イテレーションでそれぞれ苦手なチームが出現したらいつまで経っても終わらない．  
そこで，最強チームを複数選び出すようアルゴリズムを改良する  
<br><br>
S-TeamTrainingアルゴリズム
1. 最初に$S$個のチームをランダムで作り，$T_1, T_2, ..., T_S$とする
2. チーム$T_1, T_2, ..., T_S$のいずれかが勝利することが連続で$K$回起こるまで次を繰り返す
    1. $T_1, T_2, ..., T_S$がそれぞれランダムに選ばれた対戦相手$T_N$とバトルを行う
    2. ランダムバトルで$T_1, T_2, ..., T_S$のうち，$\frac{S}{R}$のチーム($R>0$)が，$T_N$に勝てなかった場合
        1. $T_1, T_2, ..., T_S$で総当たり戦を行う
        2. $T_1, T_2, ..., T_S$で最も勝利回数が少なかったチームを$T_N$と入れ替える
        3. 連勝カウントをリセットする
3. $T_1, T_2, ..., T_S$を出力する

<br>
- アルゴリズムのいいところ
    - Kの値を大きくしても，Sの値を一緒に大きくすれば現実的な時間で解が求まりやすくなる
- アルゴリズムのよくないところ
    - 実装がやや複雑
    - Rを大きくすると時間がかかる
    - 「最強」は決められない．

In [31]:
def pteams(teams):
    for i, team in enumerate(teams):
        print(f"team{i+1}")
        pteam(team)

def domestic_battle(teams, battle):
    teams_win_counts = []
    for t1 in range(len(teams)):
        team_win_count = 0
        for t2 in range(t1+1, len(teams)):
            (t1_win, t2_win) = team_battle(teams[t1], teams[t2], battle=battle)
            if t1_win > t2_win: team_win_count += 1
        teams_win_counts.append(team_win_count)

    return teams_win_counts
        
def s_team_training(s=3, k=10, r=1, initial_team=None, battle_num=10, battle=SpeedBattle, verbose=False):
    pokeidx = range(len(pokeList))
    get_team = lambda : [pokeList[p] for p in random.sample(pokeidx, 6)]
    
    if initial_team is not None:
        teams = [initial_team[i] for i in range(s)]
    else:
        teams = [get_team() for i in range(s)]

    if verbose: 
        print("======== Initial Teams ========")
        pteams(teams)

    consecutive_win_num = 0
    team_count = 1
    while consecutive_win_num < k:
        
        rival = [pokeList[p] for p in random.sample(pokeidx,6)]
        
        results = []
        for team in teams:
            (team_win, rival_win) = team_battle(team, rival, battle=battle, battle_num=battle_num)
            results.append(team_win <= rival_win)
            
        
        if sum(results) >= round(s / r):
            
            teams_win_counts = domestic_battle(teams, battle)
            
            teams[np.argmin(teams_win_counts)] = rival

            if verbose:
                print(f"======== {team_count}-th New Teams =======")
                pteams(teams)

            consecutive_win_num = 0
            team_count += 1
        else: 
            consecutive_win_num += 1
    
    if verbose:
        print(f"======== {team_count-1}-th teams are the strongest teams ========")
        pteams(teams)

    return teams, team_count-1

In [32]:
k = 10
start_time = time.time()
strongest_teams, team_count = s_team_training(k=k, battle=pf.battle)
end_time = time.time()

print(f"Elapsed Time for s-Team Training Algorithm of k={k}: {end_time-start_time}[sec] \n")
print(f"======== {team_count}-th teams are the strongest teams ========")
pteams(strongest_teams)

Elapsed Time for s-Team Training Algorithm of k=10: 0.6461648941040039[sec] 

======== 1-th teams are the strongest teams ========
team1
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 106], variety=['Psychic'])
Monster(name='Weezing', index=109, attack=90, defense=120, speed=85, hitpoint=[65, 65], variety=['Poison'])
Monster(name='Starmie', index=120, attack=75, defense=85, speed=100, hitpoint=[60, 60], variety=['Water', 'Psychic'])
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])
Monster(name='Magikarp', index=128, attack=10, defense=55, speed=20, hitpoint=[20, 20], variety=['Water'])

team2
Monster(name='Machop', index=65, attack=80, defense=50, speed=35, hitpoint=[70, -48], variety=['Fighting'])
Monster(name='Squirtle', index=6, attack=48, defense=65, speed=50, hitpo

保存版
Elapsed Time for s-Team Training Algorithm of k=10000: 2645.9579594135284[sec]   
  
======== 80-th teams are the strongest teams ========  
team1  
Monster(name='Hypno', index=96, attack=73, defense=70, speed=115, hitpoint=[85, -11], variety=['Psychic'])  
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])  
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])  
Monster(name='Mr.Mime', index=121, attack=45, defense=65, speed=100, hitpoint=[40, 40], variety=['Psychic', 'Fairy'])  
Monster(name='Poliwrath', index=61, attack=85, defense=95, speed=70, hitpoint=[90, 90], variety=['Water', 'Fighting'])  
Monster(name='Haunter', index=92, attack=50, defense=45, speed=115, hitpoint=[45, 45], variety=['Ghost', 'Poison'])  
<br>
team2  
Monster(name='Magmar', index=125, attack=95, defense=57, speed=85, hitpoint=[65, 48], variety=['Fire'])  
Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock'])  
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])  
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])  
Monster(name='Snorlax', index=142, attack=110, defense=65, speed=65, hitpoint=[160, 160], variety=['Normal'])  
Monster(name='Paras', index=45, attack=70, defense=55, speed=55, hitpoint=[35, 35], variety=['Bug', 'Grass'])  
<br>
team3  
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])  
Monster(name='Hitmonlee', index=105, attack=120, defense=53, speed=35, hitpoint=[50, 50], variety=['Fighting'])  
Monster(name='Golem', index=75, attack=110, defense=130, speed=55, hitpoint=[80, 80], variety=['Rock', 'Ground', 'Electric'])  
Monster(name='Dodrio', index=84, attack=110, defense=70, speed=60, hitpoint=[60, 60], variety=['Normal', 'Flying'])  
Monster(name='Mr.Mime', index=121, attack=45, defense=65, speed=100, hitpoint=[40, 40], variety=['Psychic', 'Fairy'])  
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 95], variety=['Water', 'Flying'])  

In [33]:
domestic_battle(strongest_teams, SpeedBattle)

[2, 0, 0]

In [34]:
results = np.zeros(3, dtype='i')
for i in range(100):
    results += domestic_battle(strongest_teams, pf.battle)
print(results)

[200  68   0]


team0やっぱり強いみたい

In [35]:
strongest_teams[0]

[Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, -32], variety=['Psychic']),
 Monster(name='Weezing', index=109, attack=90, defense=120, speed=85, hitpoint=[65, 65], variety=['Poison']),
 Monster(name='Starmie', index=120, attack=75, defense=85, speed=100, hitpoint=[60, -13], variety=['Water', 'Psychic']),
 Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice']),
 Monster(name='Rhydon', index=111, attack=130, defense=120, speed=45, hitpoint=[105, 105], variety=['Ground', 'Rock']),
 Monster(name='Magikarp', index=128, attack=10, defense=55, speed=20, hitpoint=[20, 20], variety=['Water'])]

In [36]:
normal_result_team = normal_ranking[:6]
pteam(normal_result_team)
team_battle(normal_result_team, strongest_teams[1], battle_num=1000)

Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, -25], variety=['Water', 'Flying'])
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 36], variety=['Psychic'])
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 71], variety=['Dragon', 'Flying'])
Monster(name='Mew', index=150, attack=100, defense=100, speed=100, hitpoint=[100, -1], variety=['Psychic'])



(500, 0)

それでもやっぱり最強チームには勝てない

初期チームを最強チームにしてみる？

In [37]:
s = 3
k = 1000
initial_team = [normal_result_team.copy() for i in range(s)]
for team in initial_team: random.shuffle(team)
start_time = time.time()
strongest_teams, team_count = s_team_training(k=k, initial_team=initial_team, battle=pf.battle)
end_time = time.time()

print(f"Elapsed Time for s-Team Training Algorithm of k={k}: {end_time-start_time}[sec] \n")
print(f"======== {team_count}-th teams are the strongest teams ========")
pteams(strongest_teams)

Elapsed Time for s-Team Training Algorithm of k=1000: 11.526853799819946[sec] 

======== 0-th teams are the strongest teams ========
team1
Monster(name='Mew', index=150, attack=100, defense=100, speed=100, hitpoint=[100, 100], variety=['Psychic'])
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=85, hitpoint=[50, 50], variety=['Water', 'Ice'])
Monster(name='Gyarados', index=129, attack=125, defense=79, speed=100, hitpoint=[95, 93], variety=['Water', 'Flying'])
Monster(name='Dragonite', index=148, attack=134, defense=95, speed=100, hitpoint=[91, 91], variety=['Dragon', 'Flying'])
Monster(name='Mewtwo', index=149, attack=110, defense=90, speed=154, hitpoint=[106, 106], variety=['Psychic'])

team2
Monster(name='Kingler', index=98, attack=130, defense=115, speed=50, hitpoint=[55, 55], variety=['Water'])
Monster(name='Cloyster', index=90, attack=95, defense=180, speed=8

N時間かけてアルゴリズムを回してみても結局最強チームは最強チームっぽい  

## 同じポケモンを編成可能
やっパルシェン？  
しかしパルシェン対策にサイドンを入れる必要ありそう

## 相手のポケモンがわかっている時
- とりあえず一番強いやつを重要視したほうが良いかも
- 大体パルシェンで勝てる気がする
- 相手一体一体に対して，一番勝率の高いポケモンを6体，同じ順番でぶつけてやる
- 全員相手しているのでは大変なので，有利なタイプ，相手にダメージを与えられることがわかっているポケモンから選ぶ
    - 攻撃力，防御力，HPによる指標を作っても良いかも
    - 同率はnormal_ranking順
    ポケモンそれぞれに天敵となるポケモンの表をあらかじめ作っておけば定数オーダーで作れる

## 同じポケモンを編成可能かつ相手のポケモンがわかっている時
合わせ技すればいいのかな